<h1> Generating a PUMA test set. </h1>

In [1]:
import pandas as pd
import pylab as pl

In [2]:
pl.rcParams.update({'font.size':22,
                     'figure.figsize':(12,6)})

In [3]:
# Load data with puma index.
historical = pd.read_csv('../data/citibike/dock_historical/historical_data_puma.csv', index_col=0)

/home/mark/anaconda3/envs/inapinch/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
historical = historical.query('_long > -74.03') # Remove NJ.
historical = historical.query('time >= "2018-01"') # Only consider 2018.
historical.PUMA = historical.PUMA.astype(int)   # PUMA as an int.
historical.time = pd.to_datetime(historical.time) # time as as pd.datetime obj.

<h2> Correcting time. </h2>

In [5]:
historical.PUMA.unique().shape[0]

18

In [6]:
puma = historical.query('PUMA == 3810')

In [7]:
tot_docks = puma.set_index('time').groupby('time')['tot_docks'].sum()

In [8]:
puma.dock_id.unique().shape[0]

83

In [9]:
tot_stations = puma.set_index('time').groupby('time')['dock_id'].count()

In [10]:
puma.loc[:, 'dayofyear'] = puma.time.dt.dayofyear
puma.loc[:, 'dayofweek'] = puma.time.dt.dayofweek
puma.loc[:, 'hour'] = puma.time.dt.hour
puma = puma.drop(columns=['PUMA', 'status_key', 'dock_name', 'time'])

/home/mark/anaconda3/envs/inapinch/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/mark/anaconda3/envs/inapinch/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
temp = puma.groupby(['dock_id', 'dayofweek', 'dayofyear', 'hour'])['avail_bikes', 'avail_docks', 'in_service'].agg({'avail_bikes':'mean', 'avail_docks':'mean', 'in_service':'min'}).groupby(['dayofyear', 'dayofweek', 'hour']).sum().reset_index()

In [12]:
temp.loc[:, 'time'] = pd.to_datetime('2018-' + temp.dayofyear.apply(lambda x: '{:03}'.format(x)) + ' ' + temp.hour.apply(lambda x: '{:02}'.format(x)), format='%Y-%j %H')
temp = temp.set_index('time')
ix = pd.date_range(temp.index[0], temp.index[-1], freq="H")

In [13]:
temp = temp.reindex(ix)

In [14]:
temp['avail_bikes-1'] = temp.avail_bikes.shift(-1)
temp['avail_bikes-2'] = temp.avail_bikes.shift(-2)
temp['avail_bikes-3'] = temp.avail_bikes.shift(-3)
temp['avail_bikes-4'] = temp.avail_bikes.shift(-4)
temp['avail_bikes-5'] = temp.avail_bikes.shift(-5)

In [15]:
temp['avail_docks-1'] = temp.avail_docks.shift(-1)
temp['avail_docks-2'] = temp.avail_docks.shift(-2)
temp['avail_docks-3'] = temp.avail_docks.shift(-3)
temp['avail_docks-4'] = temp.avail_docks.shift(-4)
temp['avail_docks-5'] = temp.avail_docks.shift(-5)

In [18]:
tot_docks = tot_docks.resample('D').mean().round().reindex(ix).fillna(method='ffill').astype('int')

In [19]:
tot_stations = tot_stations.resample('D').mean().round().reindex(ix).fillna(method='ffill').astype('int')

In [20]:
temp.loc[:, 'tot_docks'] = tot_docks
temp.loc[:, 'tot_stations'] = tot_stations

In [22]:
temp.to_csv('puma_3810.csv')